In [ ]:
import pandas   as pd
import polars   as pl
import numpy    as np
import networkx as nx
import time
import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()

In [ ]:
def linkNodeShortest(df, 
                     relationships, 
                     pairs, 
                     view_path_index=1, 
                     use_digraph=False,
                     y_path_gap=15,
                     x_ins=10,
                     y_ins=10,
                     txt_h=10,
                     w=512):
    return RTLinkNodeShortest(rt_self=rt, df=df, relationships=relationships, pairs=pairs, 
                              view_path_index=view_path_index, 
                              use_digraph=use_digraph, 
                              y_path_gap=y_path_gap, x_ins=x_ins, y_ins=y_ins, txt_h=txt_h, w=w)

class RTLinkNodeShortest(object):
    def __init__(self, rt_self, **kwargs):
        self.rt_self       = rt_self
        self.df            = kwargs['df']
        self.relationships = kwargs['relationships']    # [('fm','to'), (('fm1','fm2'),('to1','to2'))]
        self.pairs         = kwargs['pairs']            # [('node_0', 'node_1'), ('node_2', 'node_3'), ...]
        self.vpi           = kwargs['view_path_index']  # path index for the centered view
        self.use_digraph   = kwargs['use_digraph']      # use a directed graph
        self.y_path_gap    = kwargs['y_path_gap']
        self.x_ins         = kwargs['x_ins']
        self.y_ins         = kwargs['y_ins']
        self.txt_h         = kwargs['txt_h']
        self.w             = kwargs['w']
        self.time_lu       = {}

        # If either from or to are tuples, concat them together... // could improve a little by ensuring any same tuples are not created more than once
        _ts_ = time.time()
        new_relationships = []
        for i in range(len(self.relationships)):
            _fm_ = self.relationships[i][0]
            if type(_fm_) == list or type(_fm_) == tuple:
                new_fm = f'__fmcat{i}__'
                self.df = self.rt_self.createConcatColumn(self.df, _fm_, new_fm)
                _fm_ = new_fm
            _to_ = self.relationships[i][1]
            if type(_to_) == list or type(_to_) == tuple:
                new_to = '__tocat{i}__'
                self.df = self.rt_self.createConcatColumn(self.df, _to_, new_to)
                _to_ = new_to
            if len(self.relationships[i]) == 2: new_relationships.append((_fm_,_to_))
            else:                               new_relationships.append((_fm_,_to_,self.relationships[i][2]))
        self.relationships = new_relationships
        self.time_lu['concat_columns'] = time.time() - _ts_

        self.node_size_px = 4

    # def _repr_svg_(self):
    def _repr_svg_(self):
        return self.renderSVG()

    # def renderSVG(self):
    def renderSVG(self):
        svg = []        
        y_base = self.y_ins
        for _pair_ in self.pairs:
            g = self.rt_self.createNetworkXGraph(self.df, self.relationships, use_digraph=self.use_digraph)
            p = nx.shortest_path(g, _pair_[0], _pair_[1])
            y_top       = y_base
            y_base     += self.y_path_gap*len(p)
            x_path_gap  = (self.w - 2*self.x_ins)/(len(p)-1)
            node_to_xy  = {}
            for i in range(len(p)-1):
                n0, n1 = p[i], p[i+1]
                x0 = self.x_ins + x_path_gap*i
                node_to_xy[n0] = (x0, y_base)
                x1 = self.x_ins + x_path_gap*(i+1)
                node_to_xy[n1] = (x1, y_base)
                svg.append(f'<line x1="{x0}" y1="{y_base}" x2="{x1}" y2="{y_base}" stroke="black" stroke-width="2" />')
            for _node_ in node_to_xy:
                svg.append(f'<circle cx="{node_to_xy[_node_][0]}" cy="{node_to_xy[_node_][1]}" r="{self.node_size_px}" stroke="black" stroke-width="2" />')
            y_bot = y_base + self.y_path_gap*len(p)

            if self.vpi > 0 and self.vpi < len(p)-1:
                node_center = p[self.vpi]
                svg.append(f'<line x1="{node_to_xy[node_center][0]}" y1="{y_top}" x2="{node_to_xy[node_center][0]}" y2="{y_bot}" stroke="gray" stroke-width="0.5" stroke-dasharray="1 5 1" />')
                for i in range(len(p)-1): g.remove_edge(p[i],p[i+1])
            offset = 1
            while offset < len(p):
                j, y = self.vpi - offset, y_base - self.y_path_gap*offset
                if j > 0:
                    try:    pp = nx.shortest_path(p[j],p[self.vpi])
                    except: pp = None
                    if pp is not None:
                        svg.append(self.rt_self.svgText(f'{len(pp)}', node_to_xy[p[self.vpi]][0] - x_path_gap/4, y, self.txt_h, anchor='end'))
                    else:
                        svg.append(self.rt_self.svgText('0',          node_to_xy[p[self.vpi]][0] - x_path_gap/4, y, self.txt_h, anchor='end'))
                j = self.vpi + offset
                if j < len(p)-1:
                    try:    pp = nx.shortest_path(p[self.vpi],p[j])
                    except: pp = None
                    if pp is not None:
                        svg.append(self.rt_self.svgText(f'{len(pp)}', node_to_xy[p[self.vpi]][0] + x_path_gap/4, y, self.txt_h, anchor='start'))
                    else:
                        svg.append(self.rt_self.svgText('0',          node_to_xy[p[self.vpi]][0] + x_path_gap/4, y, self.txt_h, anchor='start'))
                offset += 1

            y_base = y_bot+self.y_path_gap
        y_base += self.y_ins
        svg.insert(0, f'<svg width="{self.w}" height="{y_base}">')
        svg.append('</svg>')
        return ''.join(svg)

df = pd.DataFrame({'fm':'a b c d e f g h i j k a  a0 a1 a2 a3 a4 a5 a6 a7 a8 a9 a10 a11'.split(),
                   'to':'b c d e f g h i j k l a0 a1 a2 a3 a4 a5 a6 a7 a8 a9 a10 a11 l' .split()})
linkNodeShortest(df, relationships=[('fm','to')], pairs=[('a','l')], view_path_index=5)